
Italy Energy System Optimization - Brownfield Analysis with ETS1 and ETS2
==========================================================================

This script creates an energy system optimization model for Italy as a whole system,
studying the effects of ETS1 (EU Emissions Trading System) and ETS2 (new ETS for 
buildings and transport) on the optimal design and operation of energy technologies.

The analysis includes:
1. Baseline optimization without ETS
2. Optimization with ETS1 only
3. Optimization with both ETS1 and ETS2
4. Comparative analysis of results

In [89]:
import adopt_net0 as adopt
import json
from pathlib import Path
import os
import pandas as pd
import numpy as np

# Create folders 
results_data_path = Path("./userData")
results_data_path.mkdir(parents=True, exist_ok=True)

# create input data path and optimization templates 
input_data_path = Path("./macro_decarbonisation")
input_data_path.mkdir(parents=True, exist_ok=True)

# Create template input JSONs
adopt.create_optimization_templates(input_data_path)

Files already exist: macro_decarbonisation\Topology.json macro_decarbonisation\ConfigModel.json


In [90]:
results_data_path = Path("./results_macroperspective")
scenario_name = "baseline_no_ETS"

In [91]:
# Load json template
with open(input_data_path / "Topology.json", "r") as json_file:
    topology = json.load(json_file)
# Nodes
topology["nodes"] = ["northwest", "northeast", "center", "south", "islands"]
# Carriers: The Carries/ Vectors we have in the CGE model are gas, oil, electricity 
topology["carriers"] = ["electricity", "heat", "gas", "hydrogen"]
# Investment periods:
topology["investment_periods"] = ["period1"]
# Save json template
with open(input_data_path / "Topology.json", "w") as json_file:
    json.dump(topology, json_file, indent=4)

In [92]:
# load json template
with open(input_data_path / "ConfigModel.json", "r") as json_file:
    configuration = json.load(json_file)
# Set time aggregation settings:
configuration["optimization"]["typicaldays"]["N"]["value"] = 30 
configuration["optimization"]["typicaldays"]["method"]["value"] = 1
# Set MILP gap
configuration["solveroptions"]["mipgap"]["value"] = 0.02
# save json template
with open(input_data_path / "ConfigModel.json", "w") as json_file:
    json.dump(configuration, json_file, indent=4)

In [93]:
adopt.create_input_data_folder_template(input_data_path)

# Define node locations based on actual Italian regional energy centers (2021 data)
# Actual coordinates of major Italian energy hubs (based on Terna grid structure)
node_location = pd.read_csv(input_data_path / "NodeLocations.csv", sep=';', index_col=0, header=0)
node_lon = {'northwest': 9.2, 'northeast': 11.9,'center': 12.5, 'south': 14.8, 'islands': 12.5}  #longitude in degrees
node_lat = {'northwest': 45.4, 'northeast': 45.5, 'center': 42.8, 'south': 40.8, 'islands': 38.0}  # latitude in degrees
node_alt = {'northwest': 120, 'northeast': 50, 'center': 250, 'south': 200, 'islands': 0}    # Elevation in meters #this neeed also needed to be changhe to 0 the alti. of Islands.
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    node_location.at[node, 'lon'] = node_lon[node]
    node_location.at[node, 'lat'] = node_lat[node]
    node_location.at[node, 'alt'] = node_alt[node]

node_location = node_location.reset_index()
node_location.to_csv(input_data_path / "NodeLocations.csv", sep=';', index=False)

adopt.show_available_technologies()

CO2Railway
CO2Ship
CO2Truck
CO2_Pipeline
electricityOffshore
electricityOnshore
electricitySimple
heat
hydrogenPipelineOffshore
hydrogenPipelineOnshore
hydrogenRailway
hydrogenShip
hydrogenSimple
hydrogenTruck
CO2_Compressor
MEA_large
MEA_medium
MEA_small
DAC_Adsorption
dac_adsorption_performanc
Boiler_El
Boiler_Industrial_NG
Boiler_Small_H2
Boiler_Small_NG
Furnace_H2
Furnace_NG
HeatPump_AirSourced
HeatPump_GroundSourced
HeatPump_SeaWaterSourced
HeatPump_WaterSourced
Hydro_Reservoir
PumpedHydro_Closed
PumpedHydro_Open
CrackerFurnace
CrackerFurnace_Electric
EthyleneCompression
EthyleneCompression_Electric
EthyleneSeparation
SteamReformer
SteamReformer_CCS
CombinedCycle_fixed_size
GasTurbine_H2_10
GasTurbine_H2_100
GasTurbine_H2_250
GasTurbine_H2_400
GasTurbine_NG_10
GasTurbine_NG_100
GasTurbine_NG_250
GasTurbine_NG_400
GasTurbine_simple
GasTurbine_simple_CCS
SteamTurbine
GT_fitting_dat
HP_fitting_dat
MP_fitting_dat
Photovoltaic
SolarThermal
WindTurbine_Offshore_11000
WindTurbine_Offshor

In [94]:
# Add required technologies for node 'North' (We will be changing this and also adding new technologies like CCS)      (Waste-to-energy)

#Northwest technological configuration
with open(input_data_path / "period1" / "node_data" / "northwest" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2"]           
technologies["existing"] = {"GasTurbine_simple": 12000, "Boiler_Small_NG": 4500, "Photovoltaic": 3200, "WindTurbine_Onshore_4000": 800}    

with open(input_data_path / "period1" / "node_data" / "northwest" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# NorthEast technological configuration
with open(input_data_path / "period1" / "node_data" / "northeast" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2"]           
technologies["existing"] = {"GasTurbine_simple": 8500, "Boiler_Small_NG": 3800, "Photovoltaic": 2800, "WindTurbine_Onshore_4000": 600}      

with open(input_data_path / "period1" / "node_data" / "northeast" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


#Center technological configuration
with open(input_data_path / "period1" / "node_data" / "center" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "Storage_H2"]           
technologies["existing"] = {"GasTurbine_simple": 9200, "Boiler_Small_NG": 3200, "Photovoltaic": 3800, "WindTurbine_Onshore_4000": 1400}     

with open(input_data_path / "period1" / "node_data" / "center" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# Add required technologies for node 'south'
with open(input_data_path / "period1" / "node_data" / "south" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "WindTurbine_Offshore_9500", "Storage_H2"]  #WindTurbine_offshore  and also municipality boilers
technologies["existing"] = {"GasTurbine_simple": 7800, "Boiler_Small_NG": 2400, "Photovoltaic": 6200, "WindTurbine_Onshore_4000": 3800}  #MWth and MWe

with open(input_data_path / "period1" / "node_data" / "south" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


#ISlands technological configuration
with open(input_data_path / "period1" / "node_data" / "islands" / "Technologies.json", "r") as json_file:
    technologies = json.load(json_file)
technologies["new"] = ["HeatPump_AirSourced", "Storage_Battery", "Photovoltaic", "WindTurbine_Onshore_4000", "WindTurbine_Offshore_9500", "Storage_H2"]  #WindTurbine_offshore  and also municipality boilers
technologies["existing"] = {"GasTurbine_simple": 3200, "Boiler_Small_NG": 800, "Photovoltaic": 2600, "WindTurbine_Onshore_4000": 2100} #MWth and MWe

with open(input_data_path / "period1" / "node_data" / "islands" / "Technologies.json", "w") as json_file:
    json.dump(technologies, json_file, indent=4)


# Copy over technology files
adopt.copy_technology_data(input_data_path)

In [95]:
adopt.show_available_networks()

In [81]:
# Add networks (here we will add an onshore electricity network, liket the one in the case study as an example)
with open(input_data_path / "period1" / "Networks.json", "r") as json_file:
    networks = json.load(json_file)
networks["new"] = ["electricityOnshore"]
networks["existing"] = ["electricityOnshore"]

with open(input_data_path / "period1" / "Networks.json", "w") as json_file:
    json.dump(networks, json_file, indent=4)

In [82]:
# === Make a new folder for the existing network (We can also create the dataframe for this and import it.)
os.makedirs(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore", exist_ok=True)

print("Existing network")

# === Connection (Existing)
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv", sep=";", index_col=0)
connection.loc["northwest", "northeast"] = 1
connection.loc["northeast", "northwest"] = 1
connection.loc["northwest", "center"] = 1
connection.loc["center", "northwest"] = 1
connection.loc["northeast", "center"] = 1
connection.loc["center", "northeast"] = 1
connection.loc["center", "south"] = 1
connection.loc["south", "center"] = 1
connection.loc["south", "islands"] = 1  # Sicily connection
connection.loc["islands", "south"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "connection.csv")

# === Distance (Existing)
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv", sep=";", index_col=0)
distance.loc["northwest", "northeast"] = 350  # milan-venice corridor 
distance.loc["northeast", "northwest"] = 350
distance.loc["northwest", "center"] = 450    # Milan-Rome corridor 
distance.loc["center", "northwest"] = 450
distance.loc["northeast", "center"] = 420     # Venice-Rome corridor
distance.loc["center", "northeast"] = 420
distance.loc["center", "south"] = 380         # Rome-Naples corridor
distance.loc["south", "center"] = 380
distance.loc["south", "islands"] = 180        #sicily cables (SA.PE.I + SA.CO.I)
distance.loc["islands", "south"] = 180
distance.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "distance.csv")

# === Size (Existing)
size = pd.read_csv(input_data_path / "period1" / "network_topology" / "existing" / "size.csv", sep=";", index_col=0)
size.loc["northwest", "northeast"] = 6500   # Po Valley 380kV lines
size.loc["northeast", "northwest"] = 6500
size.loc["northwest", "center"] = 8200     # North-Central 380kV corridor
size.loc["center", "northwest"] = 8200
size.loc["northeast", "center"] = 5800     # Adriatic corridor
size.loc["center", "northeast"] = 5800
size.loc["center", "south"] = 6800        # Central-South lines
size.loc["south", "center"] = 6800
size.loc["south", "islands"] = 1000       # Sicily cables (SA.PE.I 500MW + SA.CO.I 500MW)
size.loc["islands", "south"] = 1000
size.to_csv(input_data_path / "period1" / "network_topology" / "existing" / "electricityOnshore" / "size.csv", sep=";")
print("Size:", size)

# Delete the original template
os.remove(input_data_path / "period1" / "network_topology" / "existing" / "size.csv")


print("New network")
# === Make a new folder for the new network
os.makedirs(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore", exist_ok=True)

# === Max Size Arc (New)
arc_size = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv", sep=";", index_col=0)
arc_size.loc["northwest", "northeast"] = 10000  # Po Valley reinforcement
arc_size.loc["northeast", "northwest"] = 10000
arc_size.loc["northwest", "center"] = 12000   # North-Central reinforcement
arc_size.loc["center", "northwest"] = 12000
arc_size.loc["northeast", "center"] = 8000  # Adriatic reinforcement
arc_size.loc["center", "northeast"] = 8000
arc_size.loc["center", "south"] = 10000    # Central-South reinforcement
arc_size.loc["south", "center"] = 10000
arc_size.loc["south", "islands"] = 2000    # Additional island connections
arc_size.loc["islands", "south"] = 2000
arc_size.loc["northwest", "south"] = 8000  # Direct North-South bypass
arc_size.loc["south", "northwest"] = 8000
arc_size.loc["northeast", "south"] = 7000  # Northeast-South direct
arc_size.loc["south", "northeast"] = 7000
arc_size.loc["center", "islands"] = 5000   # Central-Islands direct
arc_size.loc["islands", "center"] = 5000
arc_size.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "size_max_arcs.csv", sep=";")
print("Max size per arc:", arc_size)

# === Connection (New)
connection = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "connection.csv", sep=";", index_col=0)
connection.loc["northwest", "northeast"] = 1
connection.loc["northeast", "northwest"] = 1
connection.loc["northwest", "center"] = 1
connection.loc["center", "northwest"] = 1
connection.loc["northeast", "center"] = 1
connection.loc["center", "northeast"] = 1
connection.loc["center", "south"] = 1
connection.loc["south", "center"] = 1
connection.loc["south", "islands"] = 1
connection.loc["islands", "south"] = 1
connection.loc["northwest", "south"] = 1
connection.loc["south", "northwest"] = 1
connection.loc["northwest", "islands"] = 1
connection.loc["islands", "northwest"] = 1
connection.loc["northeast", "south"] = 1
connection.loc["south", "northeast"] = 1
connection.loc["northeast", "islands"] = 1
connection.loc["islands", "northeast"] = 1
connection.loc["center", "islands"] = 1
connection.loc["islands", "center"] = 1
connection.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "connection.csv", sep=";")
print("Connection:", connection)

# Delete connection template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "connection.csv")

# === Distance (New)
distance = pd.read_csv(input_data_path / "period1" / "network_topology" / "new" / "distance.csv", sep=";", index_col=0)
distance.loc["northwest", "northeast"] = 350   # Turin/Milan to Venice/Trieste
distance.loc["northeast", "northwest"] = 350
distance.loc["northwest", "center"] = 450      # Turin/Milan to Rome
distance.loc["center", "northwest"] = 450
distance.loc["northwest", "south"] = 750       # Turin/Milan to Naples/Bari
distance.loc["south", "northwest"] = 750
distance.loc["northwest", "islands"] = 950    # Turin/Milan to Palermo/Cagliari
distance.loc["islands", "northwest"] = 950
distance.loc["northeast", "center"] = 420      # Venice to Rome
distance.loc["center", "northeast"] = 420
distance.loc["northeast", "south"] = 680       # Venice to Naples/Bari
distance.loc["south", "northeast"] = 680
distance.loc["northeast", "islands"] = 880    # Venice to Palermo/Cagliari
distance.loc["islands", "northeast"] = 880
distance.loc["center", "south"] = 380          # Rome to Naples
distance.loc["south", "center"] = 380
distance.loc["center", "islands"] = 520        # Rome to Palermo/Cagliari
distance.loc["islands", "center"] = 520
distance.loc["south", "islands"] = 180         # Naples/Bari to Palermo/Cagliari
distance.loc["islands", "south"] = 180
distance.to_csv(input_data_path / "period1" / "network_topology" / "new" / "electricityOnshore" / "distance.csv", sep=";")
print("Distance:", distance)

# Delete distance template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "distance.csv")

# Delete size_max_arcs template
os.remove(input_data_path / "period1" / "network_topology" / "new" / "size_max_arcs.csv")

Existing network
Connection:            northwest  northeast  center  south  islands
northwest          0          1       1      0        0
northeast          1          0       1      0        0
center             1          1       0      1        0
south              0          0       1      0        1
islands            0          0       0      1        0
Distance:            northwest  northeast  center  south  islands
northwest          0        350     450      0        0
northeast        350          0     420      0        0
center           450        420       0    380        0
south              0          0     380      0      180
islands            0          0       0    180        0
Size:            northwest  northeast  center  south  islands
northwest          0       6500    8200      0        0
northeast       6500          0    5800      0        0
center          8200       5800       0   6800        0
south              0          0    6800      0     1000
isl

In [83]:
adopt.copy_network_data(input_data_path)

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "r") as json_file:
    network_data = json.load(json_file)

network_data["Economics"]["gamma2"] = 50000
network_data["Economics"]["gamma4"] = 400

with open(input_data_path / "period1" / "network_data"/ "electricityOnshore.json", "w") as json_file:
    json.dump(network_data, json_file, indent=4)

In [84]:
# Demand profiles and import constraints based on 2021 Italian energy data

# Regional distribution based on CGE model
regional_annual_demand = {
    'northwest': 8514320.4,   # MWh (Lombardy + Piedmont industrial)
    'northeast': 6808202.8,   # MWh (Veneto + Emilia industrial/agricultural)
    'center': 5209481.4,      # MWh (Lazio + Tuscany + others)
    'south': 3908721.5,       # MWh (Campania + Puglia + others)
    'islands': 1805076.4      # MWh (Sicily + Sardinia)
    }
    
# Create hourly profiles (simplified seasonal/daily patterns)
hours = np.arange(8760)
base_profile = np.ones(8760)

# We add seasonal variation (winter heating, summer cooling)
seasonal = 0.15 * np.sin(2 * np.pi * hours / 8760 - np.pi/2)

# We add daily variation (peak during day, low at night)
daily = 0.3 * np.sin(2 * np.pi * (hours % 24) / 24 - np.pi/2)
    
hourly_data = {}
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    # Electricity demand profile
    el_profile = base_profile + seasonal + daily
    el_profile = el_profile / np.mean(el_profile) * regional_annual_demand[node] / 8760
        
    # Heat demand (higher in north, seasonal pattern)
    heat_multiplier = {'northwest': 0.4, 'northeast': 0.35, 'center': 0.25, 'south': 0.15, 'islands': 0.1}
    heat_seasonal = 0.6 * np.maximum(0, -np.sin(2 * np.pi * hours / 8760 - np.pi/2))
    heat_profile = (base_profile * 0.2 + heat_seasonal) * heat_multiplier[node] * regional_annual_demand[node] / 8760
        
    hourly_data[node] = pd.DataFrame({
       'electricity': el_profile,
       'heat': heat_profile
        })

# Fill carrier demand data for each region
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    if 'hourly_data' in locals():
        el_demand = hourly_data[node]['electricity']
        heat_demand = hourly_data[node]['heat']
    else:
        el_demand = hourly_data[node].iloc[:, 1]
        heat_demand = hourly_data[node].iloc[:, ]
    
    adopt.fill_carrier_data(input_data_path, value_or_data=el_demand, columns=['Demand'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=heat_demand, columns=['Demand'], carriers=['heat'], nodes=[node])


# 2021 Italian energy import data and cross-border capacities
import_constraints = {
    'northwest': {
        'gas_limit': 15000,      # High gas imports via Alpine pipelines (TAP, etc.)
        'electricity_limit': 4000,  # Imports from France/Switzerland
        'electricity_price': 120,   # EUR/MWh average 2021
        'gas_price': 28             # EUR/MWh average 2021
    },
    'northeast': {
        'gas_limit': 8000,       # Gas from Eastern Europe
        'electricity_limit': 2000,  # Imports from Austria/Slovenia
        'electricity_price': 115,
        'gas_price': 30
    },
    'center': {
        'gas_limit': 5000,       # Limited direct gas imports
        'electricity_limit': 1000,  # Limited cross-border
        'electricity_price': 125,
        'gas_price': 32
    },
    'south': {
        'gas_limit': 12000,      # TAP pipeline, LNG terminals
        'electricity_limit': 500,   # Limited cross-border
        'electricity_price': 130,
        'gas_price': 29
    },
    'islands': {
        'gas_limit': 3000,       # LNG terminals (limited)
        'electricity_limit': 0,     # No direct imports (island systems)
        'electricity_price': 150,   # Higher island prices
        'gas_price': 35
    }
}

# Apply import constraints and pricing
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    constraints = import_constraints[node]
    
    # Set import limits (MW for electricity, MW equivalent for gas)
    adopt.fill_carrier_data(input_data_path, value_or_data=constraints['gas_limit'], 
                           columns=['Import limit'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=constraints['electricity_limit'], 
                           columns=['Import limit'], carriers=['electricity'], nodes=[node])
    
    # Set import prices (EUR/MWh)
    adopt.fill_carrier_data(input_data_path, value_or_data=constraints['gas_price'], 
                           columns=['Import price'], carriers=['gas'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=constraints['electricity_price'], 
                           columns=['Import price'], carriers=['electricity'], nodes=[node])
    
    # Set emission factors
    adopt.fill_carrier_data(input_data_path, value_or_data=0.35, 
                           columns=['Import emission factor'], carriers=['electricity'], nodes=[node])
    adopt.fill_carrier_data(input_data_path, value_or_data=0.2, 
                           columns=['Import emission factor'], carriers=['gas'], nodes=[node])

# Load climate data for renewable resource assessment
adopt.load_climate_data_from_api(input_data_path)


Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful
Importing Climate Data...
Importing Climate Data successful


In [85]:
# Run the optimization model with the baseline scenario (no ETS)
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


c:\Python\Lib\site-packages\adopt_net0\components\technologies\genericTechnologies\res.py:206: UserWarning: TurbineName not in csv, standard WindTurbine_Onshore_1500 selected.
  warnings.warn(
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


c:\Python\Lib\site-packages\adopt_net0\components\technologies\genericTechnologies\res.py:206: UserWarning: TurbineName not in csv, standard WindTurbine_Onshore_1500 selected.
  warnings.warn(
Deriving performance data for Heat Pump...


...

Best objective 1.715562564310e+09, best bound 1.696911164704e+09, gap 1.0872%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250922164929-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1696911164.7036994
  Upper bound: 1715562564.3095412
  Number of objectives: 1
  Number of constraints: 781176
  Number of variables: 617933
  Number of binary variables: 36040
  Number of integer variables: 44687
  Number of continuous variables: 537206
  Number of nonzeros: 1832414
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 49.18499994277954
  Termination condition: optimal
  Termination message: Model was solved to op

Solving model completed in 162 s


# Mathematical Formulation for Italian Regional Energy Demand Profiles

## 1. Electricity Demand Profile

For each region $r \in \{northwest, northeast, center, south, islands\}$ and hour $t \in \{0, 1, ..., 8759\}$:

### Base Formula:
$$D_{r,t}^{el} = \frac{D_r^{annual}}{8760} \cdot [1 + S_t + H_t]$$

Where:
- $D_{r,t}^{el}$ = Electricity demand in region $r$ at hour $t$ [MWh]
- $D_r^{annual}$ = Annual electricity demand in region $r$ [MWh]
- $S_t$ = Seasonal variation component
- $H_t$ = Daily (hourly) variation component

### Seasonal Variation:
$$S_t = 0.15 \cdot \sin\left(\frac{2\pi t}{8760} - \frac{\pi}{2}\right)$$

### Daily Variation:
$$H_t = 0.3 \cdot \sin\left(\frac{2\pi (t \bmod 24)}{24} - \frac{\pi}{2}\right)$$

### Regional Annual Demands (2021 data):
- $D_{northwest}^{annual} = 8514320,4$ MWh (Lombardy + Piedmont industrial) 
- $D_{northeast}^{annual} = 6808202,8$ MWh (Veneto + Emilia industrial/agricultural)  
- $D_{center}^{annual} = 5209481,4$ MWh (Lazio + Tuscany + others)
- $D_{south}^{annual} = 3908721,5$ MWh (Campania + Puglia + others)
- $D_{islands}^{annual} = 1805076,4$ MWh (Sicily + Sardinia)

**Total Italian consumption**: $\sum_{r} D_r^{annual} = 318,000$ MWh = 318 TWh

## 2. Heat Demand Profile

### Base Formula:
$$D_{r,t}^{heat} = \frac{D_r^{annual}}{8760} \cdot \alpha_r \cdot [0.2 + S_t^{heat}]$$

Where:
- $D_{r,t}^{heat}$ = Heat demand in region $r$ at hour $t$ [MWh]
- $\alpha_r$ = Regional heat multiplier (climate-dependent)
- $S_t^{heat}$ = Seasonal heating pattern

### Regional Heat Multipliers:
- $\alpha_{northwest} = 0.4$ (High heating demand - Po Valley)
- $\alpha_{northeast} = 0.35$ (Moderate-high heating - Continental climate)
- $\alpha_{center} = 0.25$ (Moderate heating - Central Italy)
- $\alpha_{south} = 0.15$ (Low heating - Mediterranean climate)
- $\alpha_{islands} = 0.1$ (Very low heating - Island climate)

### Seasonal Heating Pattern:
$$S_t^{heat} = 0.6 \cdot \max\left(0, -\sin\left(\frac{2\pi t}{8760} - \frac{\pi}{2}\right)\right)$$

This creates a winter-peaked heating demand with zero heating in summer months.

## 3. Import Constraints and Pricing

### Gas Import Limits:
$$\sum_{t=1}^{8760} I_{r,t}^{gas} \leq L_r^{gas} \quad \forall r$$

Where:
- $I_{r,t}^{gas}$ = Gas imports to region $r$ at hour $t$ [MW]
- $L_r^{gas}$ = Annual gas import limit for region $r$ [MW]

Regional gas import limits:
- $L_{northwest}^{gas} = 15,000$ MW (Alpine pipelines)
- $L_{northeast}^{gas} = 8,000$ MW (Eastern Europe)
- $L_{center}^{gas} = 5,000$ MW (Limited direct imports)
- $L_{south}^{gas} = 12,000$ MW (TAP pipeline, LNG)
- $L_{islands}^{gas} = 3,000$ MW (LNG terminals)

### Electricity Import Limits:
$$I_{r,t}^{el} \leq L_r^{el} \quad \forall r, t$$

Where:
- $I_{r,t}^{el}$ = Electricity imports to region $r$ at hour $t$ [MW]
- $L_r^{el}$ = Electricity import capacity for region $r$ [MW]

Regional electricity import capacities:
- $L_{northwest}^{el} = 4,000$ MW (France/Switzerland)
- $L_{northeast}^{el} = 2,000$ MW (Austria/Slovenia)
- $L_{center}^{el} = 1,000$ MW (Limited cross-border)
- $L_{south}^{el} = 500$ MW (Limited cross-border)
- $L_{islands}^{el} = 0$ MW (No direct imports)

### Import Pricing:
$$C_{r,t}^{import} = P_r^{gas} \cdot I_{r,t}^{gas} + P_r^{el} \cdot I_{r,t}^{el}$$

Where regional prices (2021 averages):
- Gas prices: $P_{northwest}^{gas} = 28$, $P_{northeast}^{gas} = 30$, $P_{center}^{gas} = 32$, $P_{south}^{gas} = 29$, $P_{islands}^{gas} = 35$ EUR/MWh
- Electricity prices: $P_{northwest}^{el} = 120$, $P_{northeast}^{el} = 115$, $P_{center}^{el} = 125$, $P_{south}^{el} = 130$, $P_{islands}^{el} = 150$ EUR/MWh

## 4. Emission Factors

### Import Emissions:
$$E_{r,t}^{import} = \epsilon^{gas} \cdot I_{r,t}^{gas} + \epsilon^{el} \cdot I_{r,t}^{el}$$

Where:
- $\epsilon^{gas} = 0.2$ tCO₂/MWh (natural gas emission factor)
- $\epsilon^{el} = 0.35$ tCO₂/MWh (average EU electricity grid emission factor)

## 5. Model Integration

In the optimization model, these demand profiles become constraints:

### Energy Balance Constraint:
$$\sum_{j \in J_r} P_{j,r,t} + I_{r,t}^{el} + \sum_{k \neq r} F_{k,r,t}^{el} = D_{r,t}^{el} + \sum_{k \neq r} F_{r,k,t}^{el} \quad \forall r,t$$

Where:
- $P_{j,r,t}$ = Power generation from technology $j$ in region $r$ at time $t$
- $F_{r,k,t}^{el}$ = Electricity flow from region $r$ to region $k$ at time $t$
- $J_r$ = Set of electricity generation technologies in region $r$

### Heat Balance Constraint:
$$\sum_{j \in H_r} Q_{j,r,t} = D_{r,t}^{heat} \quad \forall r,t$$

Where:
- $Q_{j,r,t}$ = Heat generation from technology $j$ in region $r$ at time $t$
- $H_r$ = Set of heat generation technologies in region $r$

This mathematical framework ensures that the Italian energy system model captures:
1. **Realistic regional variations** in demand patterns
2. **Seasonal and daily cycles** consistent with Italian climate and lifestyle
3. **Import constraints** reflecting actual cross-border infrastructure
4. **Economic and environmental costs** of energy imports

The polices of the ETS1 (POWER SECTOR CARBON PRICING)

In [ ]:
print("Implementing ETS1 carbon pricing for power sector...")

# ETS1 carbon price: ~85 EUR/tCO2 (2021 levels)
carbon_price = np.ones(8760) * 85

# Apply ETS1 to power sector (affects electricity generation and imports)
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    carbon_cost_path = "./macro_decarbonisation/period1/node_data/" + node + "/CarbonCost.csv"
    carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
    carbon_cost_template['price'] = carbon_price
    carbon_cost_template = carbon_cost_template.reset_index()
    carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

# Run the optimization model with ETS1 scenario
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()

Implementing ETS1 carbon pricing for power sector...


--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


...

Best objective 7.514900364396e+09, best bound 7.514900364396e+09, gap 0.0000%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250918003546-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 7514900364.396171
  Upper bound: 7514900364.396171
  Number of objectives: 1
  Number of constraints: 734326
  Number of variables: 571083
  Number of binary variables: 36040
  Number of integer variables: 41087
  Number of continuous variables: 493956
  Number of nonzeros: 1767783
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 26.463000059127808
  Termination condition: optimal
  Termination message: Model was solved to opt

Solving model completed in 82 s


RUN OPTIMIZATION WITH ETS1 + ETS2 (COMPREHENSIVE CARBON PRICING)
print("Implementing ETS1 + ETS2 carbon pricing")


In [ ]:
# ETS2 carbon price: ~50 EUR/tCO2 (projected for buildings/heating)
# Combined effect: higher carbon pricing across all sectors
carbon_price = np.ones(8760) * 120  # Higher combined effect

# Apply combined ETS1+ETS2 carbon pricing
for node in ['northwest', 'northeast', 'center', 'south', 'islands']:
    carbon_cost_path = "./macro_decarbonisation/period1/node_data/" + node + "/CarbonCost.csv"
    carbon_cost_template = pd.read_csv(carbon_cost_path, sep=';', index_col=0, header=0)
    carbon_cost_template['price'] = carbon_price
    carbon_cost_template = carbon_cost_template.reset_index()
    carbon_cost_template.to_csv(carbon_cost_path, sep=';', index=False)

# Run the optimization model with ETS1+ETS2 scenario
m = adopt.ModelHub()
m.read_data(input_data_path)
m.quick_solve()


--- Reading in data ---
Input data folder has been checked successfully - no errors occurred.
Reading data from macro_decarbonisation
Topology read successfully
Model Configuration read successfully
Time series read successfully
Node Locations read successfully
Energy balance options read successfully
Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


Complete:  99.0 %Complete:  100 %


Deriving performance data for Heat Pump...


...

Best objective 8.046973318389e+09, best bound 8.046973318389e+09, gap 0.0000%


Set parameter LogFile to value ""


Set parameter LogFile to value ""
Writing results to userData\20250918004319-1


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 8046973318.38896
  Upper bound: 8046973318.38896
  Number of objectives: 1
  Number of constraints: 734326
  Number of variables: 571083
  Number of binary variables: 36040
  Number of integer variables: 41087
  Number of continuous variables: 493956
  Number of nonzeros: 1767890
  Sense: minimize
  Number of solutions: 1
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Name: Gurobi 12.02
  Status: ok
  Wallclock time: 24.47699999809265
  Termination condition: optimal
  Termination message: Model was solved to optima

Solving model completed in 85 s
